サンプルデータは下記Wikipediaより  
https://ja.wikipedia.org/wiki/YouTuber 

In [1]:
from gensim import corpora, models
from utils.preprocess import tokenize

In [2]:
save_path = './test.mm'

list_docs = [
    '2007年5月頃に、YouTubeが特に閲覧数の多いユーザーに「パートナー」（YouTubeパートナープログラム）となるよう勧誘したことが始まりとされる',
    'プログラムは当初、商業コンテンツ供給者だけに勧められたものだった）[要出典]。',
    '後にビデオ画面の隣側に広告掲載するのを条件にユーザーが利益を得ることが可能になり、2011年4月にはパートナープログラムが一般ユーザーにも開放。',
    'より多くのユーザが広告収入を得られるようになった。',
    'その後は、YouTubeで収益を得て生活する人物も現れた[7]。',
    'なお、2017年4月以降は総再生回数が10,000回以上ではないとパートナープログラムへの参加が認められなくなったが、2018年2月からはさらに過去12か月間の総再生時間が4,000時間以上、かつチャンネル登録者数が1,000人以上とより厳しい条件に引き上げられた',
    '加えてコミュニティのストライキ、スパムなどの監視が強化され、ポリシーに準拠していることの評価が厳格化されるようになった。',
    'マスメディアの露出がほとんどないにもかかわらず、年収が数億〜十数億円にも達する者・独自に握手会等のイベントを開く者もいるが、ごく少数である。'
]
indices = [i for i, _ in enumerate(range(0, len(list_docs)))]

path_dict = '/usr/lib/mecab/dic/mecab-ipadic-neologd/'
tkn = tokenize.Tokenize(path_dict=path_dict)
list_split = [tkn.tokenize_text(x) for x in list_docs]

In [3]:
# LDA
dictionary = corpora.Dictionary(list_split)
corpus = [dictionary.doc2bow(list_word) for list_word in list_split]
corpora.MmCorpus.serialize(save_path, corpus) # saving
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

num_topics = 3
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dictionary,
                             num_topics=num_topics, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)

for i in range(num_topics):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)[0:80]+'...'))

tpc_0: 0.014*"YouTube" + 0.012*"者" + 0.012*"]。" + 0.012*"も" + 0.011*"億" + 0.010*"だ" + 0...
tpc_1: 0.014*"得る" + 0.014*"多く" + 0.014*"広告収入" + 0.014*"ユーザ" + 0.013*"て" + 0.011*"よう" + ...
tpc_2: 0.014*"000" + 0.014*"," + 0.014*"以上" + 0.011*"総" + 0.011*"時間" + 0.010*"ない" + 0.0...
